In [9]:
import pandas as pd
import numpy as np
import csv
import datasets
from datasets import Dataset
from datasets import load_metric
from transformers import BertTokenizerFast 
from transformers import DataCollatorForTokenClassification 
from transformers import AutoModelForTokenClassification 

In [10]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased") 

In [11]:
label_list = ['O','B-MISC','I-MISC','B-PER','I-PER','B-ORG','I-ORG','B-LOC','I-LOC']
label_encoding_dict = {
    "B-LOC": 0,
    "B-MISC": 1,
    "B-ORG": 2,
    "B-PER": 3,
    "I-LOC": 4,
    "I-MISC": 5,
    "I-ORG": 6,
    "I-PER": 7,
    "O": 8
}

In [12]:
def convert(path):
    tokens = []
    entities = []
    with open(path, 'r',) as f:
        reader = csv.reader(f, delimiter='\t')
        temp_t = []
        temp_e = []
        next(reader)
        for line in enumerate(reader):
            if not line[1]:
                tokens.append(temp_t)
                entities.append(temp_e)
                temp_t = []
                temp_e = []
            else:
                t = [ '{}'.format(x) for x in list(csv.reader([line[1][0]], delimiter=',', quotechar='"'))[0] ]
                t[3] = int(t[3])
                temp_t.append(t[1])
                temp_e.append(t[2])
        tokens.append(temp_t)
        entities.append(temp_e)
    return Dataset.from_pandas(pd.DataFrame({'tokens': tokens, 'ner_tags': entities}))

In [13]:
train = convert("/Users/johnsonchan/Documents/Coding/Class/CMPSC 190I W/HW 4/HW4_spanish_ner_data/train.csv")

In [14]:
validate = convert("/Users/johnsonchan/Documents/Coding/Class/CMPSC 190I W/HW 4/HW4_spanish_ner_data/validation.csv")

In [15]:
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [16]:
train_tokenized = train.map(tokenize_and_align_labels, batched=True)
validate_tokenized = validate.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/8323 [00:00<?, ? examples/s]

Map:   0%|          | 0/1915 [00:00<?, ? examples/s]

In [17]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=9)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: 

In [18]:
from transformers import TrainingArguments, Trainer 
args = TrainingArguments( 
"test-ner",
evaluation_strategy = "epoch", 
learning_rate=2e-5, 
per_device_train_batch_size=16, 
per_device_eval_batch_size=16, 
num_train_epochs=3, 
weight_decay=0.01, 
) 

In [19]:
data_collator = DataCollatorForTokenClassification(tokenizer) 

In [20]:
metric = datasets.load_metric("seqeval") 

/var/folders/t_/np5sdf1d3cvf9n26y_bc24tc0000gn/T/ipykernel_4554/2709392085.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("seqeval")


In [21]:
def compute_metrics(eval_preds): 
    """
    Function to compute the evaluation metrics for Named Entity Recognition (NER) tasks.
    The function computes precision, recall, F1 score and accuracy.

    Parameters:
    eval_preds (tuple): A tuple containing the predicted logits and the true labels.

    Returns:
    A dictionary containing the precision, recall, F1 score and accuracy.
    """
    pred_logits, labels = eval_preds 
    
    pred_logits = np.argmax(pred_logits, axis=2) 
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax
    
    # We remove all the values where the label is -100
    predictions = [ 
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100] 
        for prediction, label in zip(pred_logits, labels) 
    ] 
    
    true_labels = [ 
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100] 
       for prediction, label in zip(pred_logits, labels) 
   ] 
    results = metric.compute(predictions=predictions, references=true_labels) 
    return { 
   "precision": results["overall_precision"], 
   "recall": results["overall_recall"], 
   "f1": results["overall_f1"], 
  "accuracy": results["overall_accuracy"], 
  } 

In [22]:
trainer = Trainer( 
    model, 
    args, 
   train_dataset=train_tokenized, 
   eval_dataset=validate_tokenized, 
   data_collator=data_collator, 
   tokenizer=tokenizer, 
   compute_metrics=compute_metrics 
) 

In [23]:
trainer.train() 

/Users/johnsonchan/opt/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: Placeholder storage has not been allocated on MPS device!